In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).



In [2]:
train = pd.read_csv('salary-train.csv')
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


Проведите предобработку

In [3]:
#Приведите тексты к нижнему регистру (text.lower())
train['FullDescription'] = train['FullDescription'].map(lambda x: x.lower())

In [4]:
#Замените все, кроме букв и цифр, на пробелы тексты
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [5]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков
vectorizer = TfidfVectorizer(min_df = 5)
X_train = vectorizer.fit_transform(train['FullDescription'])

In [6]:
#Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [7]:
#Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime
dictVectorizer = DictVectorizer()
X_train_ohe = dictVectorizer.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [8]:
#Объедините все полученные признаки в одну матрицу "объекты-признаки"
X_train = hstack([X_train, X_train_ohe])

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [9]:
y_train = train['SalaryNormalized']
model = Ridge(alpha=1, random_state = 241)
model.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [10]:
test  = pd.read_csv('salary-test-mini.csv')
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [11]:
test['FullDescription'] = test['FullDescription'].map(lambda x: x.lower())
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)

In [12]:
X_test = vectorizer.transform(test['FullDescription'])
X_test_ohe = dictVectorizer.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test, X_test_ohe])

In [13]:
y_test = model.predict(X_test)
y_test 

array([56570.75930423, 37139.34096467])